# Web scraping of the USJ website

This Webscraping only scrape informations about Insitutions, Admission, Financial Aide and Student Life

In [ ]:
import requests
from helpers.lookups import Url
from bs4 import BeautifulSoup
from urllib.parse import urlparse

## Gather all the links for the main pages

In [ ]:
with open("../data/links.txt", 'r') as file:
    links = [line.strip() for line in file]

In [ ]:
def get_institutions_links(url):
    """ Get the institutions links"""

    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    li_elements = soup.find_all('li', class_='rs-mega-menu mega-rs')

    # Search for the Insitution in the menu bar
    target_li_element = None
    for li_element in li_elements:
        a_element = li_element.find('a', string='Institutions')
        if a_element:
            target_li_element = li_element
            break

    href_links = set()
    if target_li_element:
        anchor_tags = target_li_element.find_all('a')
        for tag in anchor_tags:
            href = tag.get("href")
            if href and "https" in href:
                href = urlparse(href).netloc.split(".")[0]
                href_links.add(Url.USJ.value + "/" + href)

    return href_links

In [ ]:
links = set(links).union(get_institutions_links(Url.USJ.value))

In [ ]:
def gather_links(main_link, soup):
    """Gather links from the body only (ignore header and footer)"""

    links = set()

    # Find the body element and exclude the footer
    body_with_footer = soup.find("div", id="rs-about")
    if body_with_footer:
        footer_element = body_with_footer.find("footer", id="rs-footer")
        if footer_element:
            footer_element.decompose()  # Remove the footer element from the DOM

    body = body_with_footer
    # Gather links from the modified body element
    for link in body.find_all("a"):
        href = link.get("href")
        if href:
            if "http" in href or "javascript" in href:
                None
            elif href.startswith("https"):
                links.add(href)
            elif href.startswith("/"):
                new_main_link = main_link.rsplit("/", 1)[0]
                links.add(new_main_link + href)
            elif href.startswith("../"):
                new_main_link = main_link.rsplit("/", 1)[0]
                links.add(new_main_link + href[2:])
            elif href.startswith("./"):
                links.add(main_link + href[1:])
            elif ".php" in href:
                links.add(main_link + "/" + href)
            else:
                None                

    return links

In [ ]:
# link depth search
depth = 1

In [ ]:
# Gather all the links of the website
total_links = set()
for i in range(0, depth):
    current_links = links.copy()
    links = set()
    for link in current_links:
        if link[len(link) - 1] == "/":
            link = link[:-1]
        # if it is a new link we haven't scraped yet
        if link not in total_links:
            # add link to all the links
            total_links.add(link)
            try:
                response = requests.get(link)
                soup = BeautifulSoup(response.content, "html.parser")
                new_links = gather_links(link, soup)
                for n_link in new_links:
                    if n_link not in total_links and n_link not in current_links:
                        links.add(n_link)
            except Exception as e:
                print("Error Description:", str(e))

total_links = total_links.union(links)


In [ ]:
len(total_links)

In [ ]:
total_links

## Implement API

In [1]:
from links_scraping import LinksScraper
from helpers.lookups import Path

links_scraper = LinksScraper(Path.Links)
total_links = links_scraper.gather_links(1)

Error Description: 'NoneType' object has no attribute 'find_all'
Error Description: 'NoneType' object has no attribute 'find_all'
Error Description: 'NoneType' object has no attribute 'find_all'
Error Description: 'NoneType' object has no attribute 'find_all'


In [2]:
len(total_links)

615